In [1]:
import matplotlib.pyplot as plt
import cv2
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPool2D, Dropout, BatchNormalization
from tensorflow.keras.optimizers import SGD, Adam
from keras.callbacks import ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.utils import to_categorical
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from keras.models import load_model

from imutils.perspective import four_point_transform
import imutils

In [2]:
#LOAD TRAINED MODEL
model = load_model('models/model_hand_v1.h5')

In [3]:
# load the example image and convert it to grayscale
image = cv2.imread('test-images/5.jpg')
#image = cv2.resize(image, (600, 400))

# detect rectangle
# pre-process the image by resizing it, converting it to
# graycale, blurring it, and computing an edge map
#image = imutils.resize(image, height=500)
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
blurred = cv2.GaussianBlur(gray, (5, 5), 0)
edged = cv2.Canny(blurred, 50, 200, 255)

In [4]:
# find contours in the edge map, then sort them by their
# size in descending order
cnts = cv2.findContours(edged.copy(), cv2.RETR_EXTERNAL,
                        cv2.CHAIN_APPROX_SIMPLE)
cnts = imutils.grab_contours(cnts)
cnts = sorted(cnts, key=cv2.contourArea, reverse=True)
displayCnt = None

In [5]:
# loop over the contours
for c in cnts:
    # approximate the contour
    peri = cv2.arcLength(c, True)
    approx = cv2.approxPolyDP(c, 0.02 * peri, True)

    # if the contour has four vertices, then we have found
    # the thermostat display
    if len(approx) == 4:
        displayCnt = approx
        break

In [7]:
# extract the id display, apply a perspective transform
# to it
warped = four_point_transform(gray, displayCnt.reshape(4, 2))
output = four_point_transform(image, displayCnt.reshape(4, 2))

output = cv2.resize(output, (600, 400))
#cropped_image = output[65:340, 55:550]
cropped_image = output.copy()
#cv2.imshow("cropped_image", cropped_image)

In [8]:
gray = cv2.cvtColor(output, cv2.COLOR_BGR2GRAY)
gray = cv2.threshold(gray, 0, 255,cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]

kernel = np.ones((3, 3), np.uint8)
gray = cv2.erode(gray, kernel, iterations=1)

In [9]:
# detect chars
edged = cv2.Canny(gray, 50, 200, 255)

cnts = cv2.findContours(edged.copy(), cv2.RETR_TREE,
                        cv2.CHAIN_APPROX_NONE)
cnts = imutils.grab_contours(cnts)
cnts = sorted(cnts, key=cv2.contourArea, reverse=True)

In [10]:
count = 1
drawedContours = []

chars = []


# loop over the contours
for c in cnts:
    contour_area = cv2.contourArea(c)

    (x, y, w, h) = cv2.boundingRect(c)

    if not x in drawedContours:
        if contour_area > 10000 and contour_area < 30000:
            drawedContours.append(x)
            chars.append([x, y, w, h])
            cv2.rectangle(cropped_image, (x, y), (x+w, y+h), (0, 0, 255), 3)
            cv2.putText(cropped_image, str(count), (x+20, y+20),
                        cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 0, 255), 2)
            count += 1

count = 1
print(chars)

shorted_char_list = sorted(chars, key=lambda x: x[0])
print(shorted_char_list)

cropped_chars = []


for c in shorted_char_list:
    crop_img = gray[c[1]:c[1]+c[3], c[0]:c[0]+c[2]]
    #cv2.imshow("cropped_" + str(count), crop_img)
    cropped_chars.append(crop_img)
    #cv2.waitKey(0)
    count += 1
    
print(cropped_chars)


[[367, 97, 155, 233], [185, 90, 159, 230], [68, 91, 100, 233]]
[[68, 91, 100, 233], [185, 90, 159, 230], [367, 97, 155, 233]]
[array([[255, 255, 255, ..., 255, 255, 255],
       [255, 255, 255, ..., 255, 255, 255],
       [255, 255, 255, ..., 255, 255, 255],
       ...,
       [255, 255, 255, ..., 255, 255, 255],
       [255, 255, 255, ..., 255, 255, 255],
       [255, 255, 255, ..., 255, 255, 255]], dtype=uint8), array([[255, 255, 255, ..., 255, 255, 255],
       [255, 255, 255, ..., 255, 255, 255],
       [255, 255, 255, ..., 255, 255, 255],
       ...,
       [255, 255, 255, ..., 255, 255, 255],
       [255, 255, 255, ..., 255, 255, 255],
       [255, 255, 255, ..., 255, 255, 255]], dtype=uint8), array([[255, 255, 255, ..., 255, 255, 255],
       [255, 255, 255, ..., 255, 255, 255],
       [255, 255, 255, ..., 255, 255, 255],
       ...,
       [255, 255, 255, ..., 255, 255, 255],
       [255, 255, 255, ..., 255, 255, 255],
       [255, 255, 255, ..., 255, 255, 255]], dtype=uint8)]


In [56]:
# show the output images
# cv2.imshow("cropped_image", cropped_image)
# cv2.imshow("Image", image)
# cv2.imshow("Output", gray)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

In [11]:
word_dict = {0:'A',1:'B',2:'C',3:'D',4:'E',5:'F',6:'G',7:'H',8:'I',9:'J',10:'K',11:'L',12:'M',13:'N',14:'O',15:'P',16:'Q',17:'R',18:'S',19:'T',20:'U',21:'V',22:'W',23:'X', 24:'Y',25:'Z'}

predicted_text = ""

count = 1

#PREDICTION ON EXTERNAL IMAGE
for i in cropped_chars:
    img = i
    img_copy = img.copy()
    #img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    #img = cv2.resize(img, (400,440))

    img_copy = cv2.GaussianBlur(img_copy, (7,7), 0)
    #img_gray = cv2.cvtColor(img_copy, cv2.COLOR_BGR2GRAY)
    #_, img_thresh = cv2.threshold(img_gray, 100, 255, cv2.THRESH_BINARY_INV)
    _, img_thresh = cv2.threshold(img_copy, 100, 255, cv2.THRESH_BINARY_INV)
    img_final = cv2.resize(img_thresh, (28,28))
    img_final =np.reshape(img_final, (1,28,28,1))

    confidence = model.predict(img_final)
    img_pred = word_dict[np.argmax(confidence)]
    cv2.putText(img, "p_" + str(count), (20,25), cv2.FONT_HERSHEY_TRIPLEX, 0.7, color = (0,0,230))
    cv2.putText(img, "Confidence: " + "{:.2f}".format(confidence[0][np.argmax(confidence)]), (20,380), cv2.FONT_HERSHEY_DUPLEX, 1.3, color = (255,0,30))
    cv2.putText(img, "Prediction: " + img_pred, (20,200), cv2.FONT_HERSHEY_DUPLEX, 1.3, color = (255,0,0))
    cv2.imshow("p_" + str(count), img)
    predicted_text = predicted_text + str(img_pred)
    count +=1
    
    
    print("Prediction: " + img_pred)
    print("Confidence: " + "{:.2f}".format(confidence[0][np.argmax(confidence)]))
    print("--------------------------------")
    
    cv2.waitKey(0)
    

print(predicted_text)
cv2.destroyAllWindows()

Prediction: L
Confidence: 1.00
--------------------------------
Prediction: U
Confidence: 0.88
--------------------------------
Prediction: H
Confidence: 0.90
--------------------------------
LUH
